## Konvolučné neurónové siete

`keras` poskytuje priamo vrstvy pre vytvorenie konvolučných sietí, ako napr. `Conv2D` pre dvojrozmernú konvolučnú vrstvu a `MaxPooling2D` pre _pooling_ s maximálnou agregačnou funkciou. 

In [ ]:
# naimportujeme si potrebné knižnice a typy
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(12345)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# `keras` poskytuje priamo viacero demonštračných dátových množín, MNIST je množina
# obrázkov s ručne napísanými číslicami od 0 po 9
from keras.datasets import mnist
from keras.utils import np_utils, plot_model

### Predspracovanie dát

Pri predspracovaní načítame dáta do `numpy` polí a znormalizujeme ich hodnoty.

In [ ]:
# načítame trénovacie a testovacie dáta
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# máme 60000 trénovacích čierno-bielych obrázkov s rozmerom 28x28 pixelov
# a 10000 testovacích
print(x_train.shape, x_test.shape)

In [ ]:
# zobrazíme si prvý trénovací príklad ako obrázok
plt.imshow(x_train[0], cmap='gray')

In [ ]:
# dáta si musíme preusporiadať na tenzor s rozmermi
# počet obrázkov x šírka x výška x počet farebných kanálov
# keďže máme čiernobiele obrázky, máme iba jeden farebný kanál
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# pôvodné hodnoty sú celé čísla od 0 (čierna farba) po 255 (biela)
# zmeníme typ na desatinné čísla a znormujeme dáta do intervalu 0-1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
# cieľový kategorický atribút je potrebné zakódovať pre výstupnú vrstvu ako binárny vektor
# pre binárne kódovanie môžete využiť pomocnú funkciu z knižnice `keras`
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

### Vytvorenie modelu

In [ ]:
# vytvoríme sekvenčný model
model = Sequential()

# 32 filtrov, veľkosť 3x3, vstupný obrázok 28x28 pixelov x1 farebný kanál
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
# 32 filtrov, veľkosť 3x3
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
# pooling, veľosť 2x2
model.add(MaxPooling2D(pool_size=(2,2)))
# dropout pre regularizáciu učenia
model.add(Dropout(0.25))

# Flatten transformuje vstup z dvojrozmernej mriežky na jednorozmerný vektor pre
# doprednú sieť
model.add(Flatten())
# dopredná sieť s jednou skrytou vrstvou - 128 neurónov
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# výstupná vrstva pre 10 tried
model.add(Dense(10, activation='softmax'))

# zobrazíme si štruktúru modelu a počet parametrov na jednotlivých vrstvách,
# všimnite si, že väčšina parametrov je na doprednej sieti
model.summary()

In [ ]:
# model si môžete zobraziť aj graficky
# plot_model(model, to_file='convolution_model.png', show_shapes=True)

### Učenie a vyhodnotenie presnosti

In [ ]:
adam = Adam()
# štandardná procedúra učenia pre klasifikáciu
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
f = model.fit(x_train, y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test))

In [ ]:
# zobrazíme si prvý testovací príklad
plt.imshow(x_test[0].reshape(28,28), cmap='gray')

In [ ]:
predictions = model.predict(x_test[:1])
# zobrazíme si predikciu pre jednotlivé triedy (číslice) pre prvý testovací príklad
print(predictions[0])
plt.bar(np.arange(10), predictions[0])
plt.xticks(np.arange(10))